# Ahead-of-Time Compilation

This cookbook describes how to radically improve performance by compiling ahead-of-time (AOT) during a build process.

本书介绍了如何通过在构建过程中编译提前（AOT）来彻底改进性能。

## Overview

An Angular application consists largely of components and their HTML templates. Before the browser can render the application, the components and templates must be converted to executable JavaScript by the Angular compiler.

Angular应用程序主要由组件及其HTML模板组成。 在浏览器渲染应用程序之前，Angular 编译器必须将组件和模板转换为可执行JavaScript。

You can compile the app in the browser, at runtime, as the application loads, using the just-in-time (JIT) compiler. This is the standard development approach shown throughout the documentation. It's great but it has shortcomings.

您可以使用即时（JIT）编译器在应用程序加载时在运行时在浏览器中编译应用程序。 这是整个文档中显示的标准开发方法。 这是伟大的，但它有缺点。

JIT compilation incurs a runtime performance penalty. Views take longer to render because of the in-browser compilation step. The application is bigger because it includes the Angular compiler and a lot of library code that the application won't actually need. Bigger apps take longer to transmit and are slower to load.

JIT 编译会导致运行时性能损失。 由于浏览器编译步骤，视图需要更长的时间才能呈现。 应用程序更大，因为它包括 Angular 编译器和应用程序实际不需要的大量库代码。 更大的应用程序需要更长的时间才能传输，并且加载速度较慢。

Compilation can uncover many component-template binding errors. JIT compilation discovers them at runtime, which is late in the process.

编译可以发现许多组件模板绑定错误。 JIT 编译在运行时发现它们，这在进程的后期。

The ahead-of-time (AOT) compiler can catch template errors early and improve performance by compiling at build time.

提前（AOT）编译器可以早期捕获模板错误，并通过在构建时编译来提高性能。

## Ahead-of-time (AOT) vs just-in-time (JIT)

There is actually only one Angular compiler. The difference between AOT and JIT is a matter of timing and tooling. With AOT, the compiler runs once at build time using one set of libraries; with JIT it runs every time for every user at runtime using a different set of libraries.

实际上只有一个 Angular 编译器。 AOT 和 JIT 之间的区别是时间和工具的问题。 使用AOT，编译器在构建时使用一组库运行一次; 使用JIT，每次运行时都会使用不同的库集来运行每个用户。

## Why do AOT compilation?

- Faster rendering

With AOT, the browser downloads a pre-compiled version of the application. The browser loads executable code so it can render the application immediately, without waiting to compile the app first.

使用 AOT，浏览器下载应用程序的预编译版本。 浏览器加载可执行代码，因此可以立即呈现应用程序，而无需等待首先编译应用程序。

- Fewer asynchronous requests

The compiler inlines external HTML templates and CSS style sheets within the application JavaScript, eliminating separate ajax requests for those source files.

编译器内置了应用程序 JavaScript 中的外部 HTML 模板和 CSS 样式表，从而消除了对这些源文件的单独的 ajax 请求。

- Smaller Angular framework download size

There's no need to download the Angular compiler if the app is already compiled. The compiler is roughly half of Angular itself, so omitting it dramatically reduces the application payload.

如果应用程序已经被编译，则不需要下载 Angular 编译器。 编译器大约是 Angular 本身的一半，因此省略它大大减少了应用程序有效负载。

- Detect template errors earlier

The AOT compiler detects and reports template binding errors during the build step before users can see them.

AOT 编译器在构建步骤中检测并报告模板绑定错误，然后用户就能看到它们。

- Better security

AOT compiles HTML templates and components into JavaScript files long before they are served to the client. With no templates to read and no risky client-side HTML or JavaScript evaluation, there are fewer opportunities for injection attacks.

AOT 将 HTML 模板和组件编译成 JavaScript 文件，直到它们被提供给客户端。 没有模板阅读，没有风险的客户端 HTML 或 JavaScript 评估，注入攻击的机会较少。

- Compile with AOT

Preparing for offline compilation takes a few simple steps. Take the Setup as a starting point. A few minor changes to the lone app.component lead to these two class and HTML files:

准备离线编译需要一些简单的步骤。 以安装为起点。 单独的app.component的一些细微变化导致这两个类和 HTML 文件：

** src/app/app.component.html **

```html
<button (click)="toggleHeading()">Toggle Heading</button>
<h1 *ngIf="showHeading">Hello Angular</h1>

<h3>List of Heroes</h3>
<div *ngFor="let hero of heroes">{{hero}}</div>
```

** src/app/app.component.ts **

```js
import { Component } from '@angular/core';
 
@Component({
  selector: 'my-app',
  templateUrl: './app.component.html'
})
export class AppComponent {
  showHeading = true;
  heroes = ['Magneta', 'Bombasto', 'Magma', 'Tornado'];
 
  toggleHeading() {
    this.showHeading = !this.showHeading;
  }
}
```

Install a few new npm dependencies with the following command:

```bash
npm install @angular/compiler-cli @angular/platform-server --save
```

You will run the `ngc` compiler provided in the `@angular/compiler-cli` npm package instead of the TypeScript compiler (`tsc`).

`ngc` is a drop-in replacement for tsc and is configured much the same way.

`ngc` requires its own tsconfig.json with AOT-oriented settings. Copy the original `src/tsconfig.json` to a file called `tsconfig-aot.json` on the project root, then modify it as follows.

** tsconfig-aot.json **

```json
{
  "compilerOptions": {
    "target": "es5",
    "module": "es2015",
    "moduleResolution": "node",
    "sourceMap": true,
    "emitDecoratorMetadata": true,
    "experimentalDecorators": true,
    "lib": ["es2015", "dom"],
    "noImplicitAny": true,
    "suppressImplicitAnyIndexErrors": true,
    "typeRoots": [
      "./node_modules/@types/"
    ]
  },

  "files": [
    "src/app/app.module.ts",
    "src/main.ts"
  ],

  "angularCompilerOptions": {
   "genDir": "aot",
   "skipMetadataEmit" : true
 }
}
```

The compilerOptions section is unchanged except for one property. Set the module to es2015. This is important as explained later in the Tree Shaking section.

What's really new is the `ngc` section at the bottom called `angularCompilerOptions`. Its genDir property tells the compiler to store the compiled output files in a new `aot` folder.

The "skipMetadataEmit" : true property prevents the compiler from generating metadata files with the compiled application. Metadata files are not necessary when targeting TypeScript files, so there is no reason to include them.

- Component-relative template URLS

The AOT compiler requires that @Component URLS for external templates and CSS files be component-relative. That means that the value of @Component.templateUrl is a URL value relative to the component class file. For example, an 'app.component.html' URL means that the template file is a sibling of its companion app.component.ts file.

While JIT app URLs are more flexible, stick with component-relative URLs for compatibility with AOT compilation.

- Compiling the application

Initiate AOT compilation from the command line using the previously installed `ngc` compiler by executing:

```bash
node_modules/.bin/ngc -p tsconfig-aot.json
```
Windows users should surround the ngc command in double quotes:

```bash
"node_modules/.bin/ngc" -p tsconfig-aot.json
```

`ngc` expects the `-p` switch to point to a `tsconfig.json` file or a folder containing a `tsconfig.json` file.

After `ngc` completes, look for a collection of NgFactory files in the aot folder. The aot folder is the directory specified as genDir in tsconfig-aot.json.

These factory files are essential to the compiled application. Each component factory creates an instance of the component at runtime by combining the original class file and a JavaScript representation of the component's template. Note that the original component class is still referenced internally by the generated factory.

> The curious can open aot/app.component.ngfactory.ts to see the original Angular template syntax compiled to TypeScript, its intermediate form.

> JIT compilation generates these same NgFactories in memory where they are largely invisible. AOT compilation reveals them as separate, physical files.

> Do not edit the NgFactories! Re-compilation replaces these files and all edits will be lost.
